#Finding an Optimal Policy to Play Chutes and Ladders Using Dynamic Programming

This notebook uses Dynamic Programming to solve the Chutes and Ladders modified game.

The game board is shown below.  Players start on Square 0 (outside the board) and move towards the goal space (State 100).  Landing at the bottom of ladder moves the player to the top, while landing at the top of a chute moves the player to the bottom square.  


<img src="https://drive.google.com/uc?export=view&id=16k2EflsluUXCPVWVgL-vyv5-IzOqWZvW" alt="Drawing" width="500"/>


At each turn, the player may choose one of four dice (Effron Dice)
- Blue: 3,3,3,3,3,3
- Black: 4,4,4,4,0,0
- Red: 6,6,2,2,2,2
- Green: 5,5,5,1,1,1

The **purpose** of this code is to determine which dice to select at each turn so as to minimize the number of steps it takes to reach the goal state.  



In [ ]:
import numpy as np

ladders = {1:38,4:14,9:31,21:42,28:84,36:44,51:67,71:91,80:100}
chutes = {16:6,48:26,49:11,56:53,62:19,64:60,87:24,93:73,95:75,98:78}

G = []
B = []
R = []
L = []

Dynamic programming uses an offline mathmatical model to calculate the expected number of moves at each possible space on the board, also known as the current state. Our goal is to create a matrix of size 101 by 101 which where each row will represent the current state and each column will represent the probability of reaching the that state from the current state. Once we have this matrix, we can use it in combination with a non-coefficient matrix to use a linear matrix solver to find our answer.

In order to calcualte the expected number of moves for each state, we must use a set of mathmatical equations which use the expected value of a future state times the probability of reaching that state to determine the value. Below are the four equations for each of the dice.

**Black Dice**

$S_{current} = 1 + S_{current + 4} * 2/3 + S_{current} * 1/3$

**Red Dice**

$S_{current} = 1 + S_{current + 6} * 1/3 + S_{current + 2} * 2/3$

**Green Dice**

$S_{current} = 1 + S_{current + 5} * 1/2 + S_{current + 1} * 1/2$

**Blue Dice**

$S_{current} = 1 + S_{current + 3}$

We can use these equations to generate the expected value for each state given the dice being used. However, before we do that we have to seperate the coefficient and non-coefficient terms. Doing so results in the following equations which will be used in our matrix:

**Black Dice**

$1 = S_{current + 4} * -2/3 + S_{current} * -1/3 + S_{current}$

**Red Dice**

$1 = S_{current + 6} * -1/3 + S_{current + 2} * -2/3 + S_{current}$

**Green Dice**

$1 = S_{current + 5} * -1/2 + S_{current + 1} * -1/2 + S_{current}$

**Blue Dice**

$1 =  -S_{current + 3} + S_{current}$

With these equations created, we can use the following functions to generate arrays of size 101 where each index represents the probability of reaching that state from the current state.

In [ ]:
def blackDice(currentState):
    """
    Generate a list of values which represent the probability of reaching future states from the
    current state using the black dice.

    Parameters:
      currentState: the current index of the array

    Return Value:
      coef: a list containing the probabilities of reaching future states
    """
    coef = [0.0] * 101        #create empty array and set current state equal to 1
    coef[currentState] = 1
    if currentState + 4 > 100:    #account for cases where next state is past the final state
        ns = 100
    else:
        ns = currentState + 4
    coef[ns] += -2/3            #fill in probabilites for each state
    coef[currentState] += -1/3
    return coef

def blueDice(currentState):
    """
    Generate a list of values which represent the probability of reaching future states from the
    current state using the blue dice.

    Parameters:
      currentState: the current index of the array

    Return Value:
      coef: a list containing the probabilities of reaching future states
    """
    coef = [0.0] * 101        #create empty array and set current state equal to 1
    coef[currentState] = 1
    if currentState + 3 > 100:  #account for cases where next state is past the final state
        ns = 100
    else:
        ns = currentState+3
    coef[ns] += -1          #fill in probabilites for each state
    return coef

def redDice(currentState):
    """
    Generate a list of values which represent the probability of reaching future states from the
    current state using the red dice.

    Parameters:
      currentState: the current index of the array

    Return Value:
      coef: a list containing the probabilities of reaching future states
    """
    coef = [0.0] * 101    #create empty array and set current state equal to 1
    coef[currentState] = 1
    if currentState + 6 > 100:  #account for cases where next state is past the final state
        ns = 100
    else:
        ns = currentState+6
    if currentState + 2 > 100:
        ns2 = 100
    else:
        ns2 = currentState+2
    coef[ns] += -1/3          #fill in probabilites for each state
    coef[ns2] += -2/3
    return coef

def greenDice(currentState):
    """
    Generate a list of values which represent the probability of reaching future states from the
    current state using the green dice.

    Parameters:
      currentState: the current index of the array

    Return Value:
      coef: a list containing the probabilities of reaching future states
    """
    coef = [0.0] * 101      #create empty array and set current state equal to 1
    if currentState + 5 > 100:  #account for cases where next state is past the final state
        ns = 100
    else:
        ns = currentState+5
    if currentState + 1 > 100:
        ns2 = 100
    else:
        ns2 = currentState+1
    coef[ns] += -1/2            #fill in probabilites for each state
    coef[ns2] += -1/2
    coef[currentState] = 1
    return coef

However, these values are not correct for each state. As mentioned earlier, the board contains chutes and ladders which will move the player to a specified space. To account for these we must go through our matrix and replace each ladder start space with an array where the probability of reaching the next state is gaurenteed with a value of 1, or negative one, based on the following equation where we use 0 instead of 1 since no additional moves are required.

**Ladder and Chute Space**

$S_{start space} = 0 + S_{end space}$

$0 = -S_{end space} + S_{start space}$

In [ ]:
def replaceLaddersandChutes(A):
    """
    Given a probability matrix replace each chute and ladder row with the proper values

    Parameters:
      A: probability matrix

    Return Value:
      A: updated probability matrix
    """
    for item in ladders:
        coef = [0.0] * 101
        coef[ladders[item]] = -1      #update the next state
        coef[item] = 1                #update the current state
        A[item] = coef

    for item in chutes:
        coef = [0.0] * 101
        coef[chutes[item]] = -1     #update the value for the next state
        coef[item] = 1              #update the current state value
        A[item] = coef

    return A

With these equations functions we are able to properly generate our matrix for each dice. In the code below we generate four arrays where each represents one of the dice. We also generate our b array which holds all of the non-coefficient values in it. All of these values are set to 1, since moving from one state should take one move, except for the chute and ladder spaces which move the player for free and the end space which results in a win.  

In [ ]:
for s in range(101):          #fill 4 arrays with probabilities of each dice type
    G.append(greenDice(s))
    L.append(blueDice(s))
    R.append(redDice(s))
    B.append(blackDice(s))


G = replaceLaddersandChutes(G)    #account for all chutes and ladders
L = replaceLaddersandChutes(L)
R = replaceLaddersandChutes(R)
B = replaceLaddersandChutes(B)

b = np.ones(101)      #generate non-coefficient matrix
b[100] = 0            #set the win state value equal to 0
for item in ladders:    #set chute and ladder spaces equal to 0
    b[item] = 0
for item in chutes:
    b[item] = 0

In [ ]:
def solve(A, b):
    """
    Take in a probability and non-coefficient vector and solve using numpy's linear matrix solver

    Parameters:
      A: probability matrix
      b: non-coefficient matrix

    Return Value:
      x: the solved matrix
    """

    A = np.array(A)
    #print(A)

    x = np.linalg.solve(A, b)
    return(x)

Our goal is to find the optimal dice to use at each state. In order to find this value, we use the following setup. This code will, using the green dice matrix as a base, go through all states in the matrix and replace it with the each of the four dice and determine which results the lowest expected number of turns. We test to see if the new dice is better than the current by seeing if the new expected value for the current state is lower than the orignal. Since we are using dynamic programming, we assume that the optimal solution from a earlier state will be correct if all later states aare optimal. Since several of these states are dependent on each other we run through this proccess several times to account for changes that occured in between runs.

In [ ]:
array = G.copy()      #create an array to test the current change
current = G.copy()    #create an array to store the current best choices
policy = ["G"] * 101  #create a policy array to save what the current best dice in.
for j in range(4):
    for i in range(101):
        array[i] = B[i]     #replace an index with a row from the black dice
        if solve(array, b)[i] < solve(current, b)[i]:   #check to see if the new dice is better than the current dice
            current[i] = B[i]
            policy[i] = "B"                              #update current and policy

        array = current.copy()      #update array to the current best options
        i -= 1

    i = 100
    while i >= 0:           #repeate the proccess with the red dice
        array[i] = R[i]
        if solve(array, b)[i] < solve(current, b)[i]:
            current[i] = R[i]
            policy[i] = "R"

        array = current.copy()
        i -= 1

    i = 100
    while i >= 0:
        array[i] = L[i]
        try:              #a loop can occur with the blue dice so we use a try to avoid any errors
            if solve(array, b)[i] < solve(current, b)[i]:   #repeate the proccess with the blue dice
                current[i] = L[i]
                policy[i] = "L"
        except:
            policy[i] = policy[i]
        array = current.copy()
        i -= 1

solution = solve(current, b)      #save the expected value matrix

After running this code we get the following solution:

In [ ]:
print(solution)

[10.58333333  8.55555556 11.11468865 11.11111111  9.52503429 10.61111111
 10.11111111 10.3002337  11.29399736  9.11111111 10.67847889 10.29399736
 10.0907636   9.67847889  9.52503429  9.0907636  10.11111111  8.52503429
  8.22222222  7.85390947  7.86728395  7.22222222  7.11728395  6.86728395
  6.61728395  6.11728395  7.15226337  7.66975309  5.11728395  9.22222222
  8.72222222  9.11111111  8.22222222  7.72222222  8.13888889  8.34722222
  6.72222222  8.88888889  8.55555556  8.22222222  7.97222222  7.72222222
  7.22222222  7.22222222  6.72222222  6.22222222  8.51592745  5.72222222
  7.15226337 10.29399736  7.01592745  4.22222222  8.80963268  8.41035411
  8.20930498  7.80963268  8.41035411  7.20930498  7.01028807  6.61179698
  6.40432099  6.01028807  7.85390947  5.40432099  6.40432099  4.81481481
  4.50925926  4.22222222  3.88888889  3.58333333  3.33333333  3.
  2.75        2.5         2.          1.75        1.5         1.
  2.70576132  3.55864198  0.          6.11728395  5.70781893  5.453

In [ ]:
for i in range(0, 100, 10):
  print(policy[i:i+10])

['G', 'G', 'R', 'R', 'G', 'B', 'L', 'R', 'R', 'G']
['L', 'R', 'L', 'R', 'L', 'R', 'G', 'R', 'L', 'R']
['L', 'G', 'R', 'G', 'B', 'L', 'R', 'G', 'G', 'B']
['R', 'R', 'B', 'L', 'R', 'G', 'G', 'R', 'R', 'R']
['G', 'R', 'L', 'B', 'L', 'R', 'B', 'B', 'G', 'G']
['G', 'G', 'R', 'R', 'L', 'R', 'G', 'R', 'L', 'R']
['L', 'R', 'G', 'R', 'G', 'R', 'R', 'R', 'R', 'R']
['R', 'G', 'L', 'B', 'R', 'G', 'B', 'L', 'R', 'G']
['G', 'L', 'R', 'R', 'R', 'L', 'R', 'G', 'R', 'R']
['R', 'R', 'L', 'G', 'R', 'G', 'B', 'L', 'G', 'G']


These two arrays show us the expected number of moves to compelte the game at each state as well as the best dice to use at each state, from 0 to 100. In the end, we expect it to take 10.58 moves on average to compelte the game of "Chutes and Ladders" using this optimal strategy. However, if we get all of the desired rolls, the above strategy will result in a win in eight moves.